<a target="_blank" href="https://colab.research.google.com/github/echosprint/TabularTransformer/blob/main/notebooks/regression.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---


**for more details about the [TabularTransformer](https://github.com/echosprint/TabularTransformer) model**,
ckeck the online **[Documents](https://echosprint.github.io/TabularTransformer/)**

---

- This notebook provides a usage example of the
  [TabularTransformer](https://github.com/echosprint/TabularTransformer)
  package.
- Hyperparameters are not tuned and may be suboptimal.

In [ ]:
%pip install git+https://github.com/echosprint/TabularTransformer.git

In [ ]:
import tabular_transformer as ttf
from tabular_transformer.tabular_loader import Task
from tabular_transformer.featurestats import FeatureStats
from tabular_transformer.tabular_dataset import TabularDataset
import torch

In [ ]:
fish_dataset_path = ttf.prepare_fish_dataset()

In [ ]:
ensure_categorical_cols = ['Species']
ensure_numerical_cols = ['Weight', 'Length1',
                         'Length2', 'Length3',
                         'Height', 'Width']

In [ ]:
fish_data_reader = ttf.DataReader(
    file_path='./data/fish/fish.csv',
    ensure_categorical_cols=ensure_categorical_cols,
    ensure_numerical_cols=ensure_numerical_cols,
    label='Width',
    header=True,
    column_names=None)

df = fish_data_reader.read().to_pandas()
print(df.head(3))

In [ ]:
split = fish_data_reader.split_data(
    {'train': 0.8, 'test': -1},
    seed=42,
    save_as='csv')

print(split)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = 'bfloat16' if torch.cuda.is_available() \
    and torch.cuda.is_bf16_supported() else 'float16'


ts = ttf.TrainSettings(wandb_log=False,
                       device=device,
                       dtype=dtype,
                       )


hp = ttf.HyperParameters(dim=32,
                         n_layers=4,
                         n_heads=4,
                         output_forward_dim=4,
                         output_hidden_dim=64)

trainer = ttf.Trainer(hp=hp, ts=ts)

In [ ]:
train_tp = ttf.TrainParameters(
    learning_rate=5e-4,
    lr_scheduler='constant',
    max_iters=300,
    loss_type='MSE',
    batch_size=16,
    output_dim=1,
    eval_interval=10,
    eval_iters=2,
    warmup_iters=5,
    validate_split=0.23,
    output_checkpoint='fish_ckpt.pt',
    input_checkpoint='fish_ckpt.pt')

trainer.train(
    data_reader=fish_data_reader(file_path=split['train']),
    tp=train_tp,
    resume=False)

In [ ]:
predictor = ttf.Predictor(checkpoint='out/fish_ckpt.pt')
predictor.predict(data_reader=fish_data_reader(file_path=split['test']),
                  save_as="fish_predictions.csv")